In [1]:
import os

In [2]:
base_dir = "C:\\Users\\Faris\\Documents\\DS-Personal Project\\chest_xray"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [3]:
os.listdir(base_dir)

['chest_xray', 'test', 'train', 'val', '__MACOSX']

In [4]:
train_normal_dir = os.path.join(train_dir, 'NORMAL')
train_pneu_dir = os.path.join(train_dir, 'PNEUMONIA')
validation_normal_dir = os.path.join(validation_dir, 'NORMAL')
validation_pneu_dir = os.path.join(validation_dir, 'PNEUMONIA')

In [5]:
print(len(os.listdir(train_normal_dir)))
print(len(os.listdir(train_pneu_dir)))

1341
3875


In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

In [8]:
img_size=[224,224]
model=tf.keras.applications.resnet50.ResNet50(input_shape=img_size + [3], weights='imagenet', include_top=False)
model.trainable = False

94773248/94765736 [==============================] - 271s 3us/step


In [9]:
last_layer = model.get_layer("conv5_block3_out")
print("last layer output shape: ", last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 2048)


In [10]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1,activation='sigmoid')(x)

In [11]:
amodel = Model(model.input,x)

In [12]:
amodel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [13]:
from tensorflow.keras.optimizers import RMSprop
amodel.compile(optimizer = RMSprop(lr=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=20,
                                                   class_mode = 'binary',
                                                   target_size = (224,224))

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       batch_size = 20,
                                                       class_mode = 'binary',
                                                       target_size = (224,224))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [15]:
history = amodel.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch = 100,
                    epochs = 10,
                    validation_steps = 50,
                    verbose = 2)

Train for 100 steps, validate for 50 steps
Epoch 1/10
100/100 - 894s - loss: 1.1519 - accuracy: 0.8595 - val_loss: 0.0364 - val_accuracy: 0.5000
